# 🎬 TMDB Interactive Movie EDA

**Author:** Vinícius Mangueira  
**Date:** May 13, 2025  
**Context:**  
This notebook demonstrates a full end-to-end Exploratory Data Analysis (EDA) workflow using the TMDB API. You’ll see everything from raw data ingestion and storage, through cleaning and transformation, to interactive visualizations that surface insights about popular movies.

---

## Objectives

1. **Extract** popular movie data directly from the TMDB API.  
2. **Persist** raw JSON responses to ensure reproducibility.  
3. **Clean** and **prepare** a pandas DataFrame ready for analysis.  
4. **Explore** rating distributions, release trends, and genre popularity.  
5. **Interact** with visualizations via widgets to support dynamic decision‑making.

---

## Notebook Structure

1. **Environment Setup** – Install dependencies and load environment variables.  
2. **Data Acquisition** – Functions and loops to fetch and save JSON payloads.  
3. **Data Cleaning** – Pandas transformations to produce a “clean” DataFrame.  
4. **EDA** – Static and interactive charts with Plotly Express.  
5. **Insights** – Key findings and patterns observed.  
6. **Deployment** – Turning this notebook into a dashboard with Voila.

> 💡 **Tip:** Use the table of contents on the side to navigate sections, and adjust widget parameters to explore different perspectives on the data.

---


**🔒 Best Practices**  
- Uses `python-dotenv` to protect API keys and keep repo clean  
- Fails early with `assert` if configuration is missing  
- Comments installation lines for seamless teammate onboarding  


In [1]:
#Import the necessary libraries
# -*- coding: utf-8 -*-
import os
import json
from pathlib import Path
import requests
from dotenv import load_dotenv
import time
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display


# Loading environment variables from .env file
# This is necessary to use the TMDB API key
load_dotenv()  
TMDB_BEARER = os.getenv('TMDB_BEARER')
assert TMDB_BEARER, "Erro: defina TMDB_BEARER no seu .env com o token de leitura (v4)"


**🔧 Engineering Rigor**  
- Abstracted API logic into a reusable function for easy testing  
- Loops through pages with on‑disk JSON snapshots for full reproducibility  
- Uses `pathlib` for cross‑platform, robust path handling  


In [2]:

# Delay between API requests (in seconds)
delay_seconds = 1  # adjust as needed

# Load the Bearer Token (v4)
load_dotenv()
TMDB_BEARER = os.getenv('TMDB_BEARER')
assert TMDB_BEARER, "Error: set TMDB_BEARER in your .env file with the v4 token"

headers = {
    'Authorization': f'Bearer {TMDB_BEARER}',
    'Content-Type': 'application/json;charset=utf-8'
}

def fetch_page(page=1):
    url = "https://api.themoviedb.org/3/movie/popular"
    params = {'language': 'en-US', 'page': page}
    r = requests.get(url, headers=headers, params=params)
    r.raise_for_status()
    return r.json()

# 1) Discover how many total pages are available
first = fetch_page(1)
total_pages = first['total_pages']
print(f"🔎 Total pages available: {total_pages}")

# 2) Set how many pages to fetch (limit)
max_pages = 50  # adjust your limit
pages_to_fetch = min(total_pages, max_pages)
print(f"📥 Will fetch pages 1 through {pages_to_fetch}")

# 3) Load already existing movie IDs from data/raw/
raw_folder = Path("data/raw")
raw_folder.mkdir(parents=True, exist_ok=True)
existing_ids = set()
for f in raw_folder.glob("popular_page_*.json"):
    with open(f, encoding="utf-8") as fh:
        data = json.load(fh)
        existing_ids.update([m['id'] for m in data])
print(f"🎬 {len(existing_ids)} movies already on disk")

# 4) Download loop with delay and avoiding duplicates
movies_raw = []
for p in range(1, pages_to_fetch + 1):
    page_data = fetch_page(p)
    time.sleep(delay_seconds)
    new_chunk = [m for m in page_data['results'] if m['id'] not in existing_ids]
    if not new_chunk:
        print(f"⚠ Page {p}: no new movies, skipping")
        continue
    # Save raw JSON
    with open(raw_folder / f"popular_page_{p}.json", "w", encoding="utf-8") as f:
        json.dump(new_chunk, f, ensure_ascii=False, indent=2)
    movies_raw.extend(new_chunk)
    existing_ids.update([m['id'] for m in new_chunk])
    print(f"✅ Page {p}: fetched {len(new_chunk)} new movies")

print(f"🎉 Total new movies fetched: {len(movies_raw)}")



🔎 Total pages available: 50316
📥 Will fetch pages 1 through 50
🎬 1000 movies already on disk
⚠ Page 1: no new movies, skipping
⚠ Page 2: no new movies, skipping
⚠ Page 3: no new movies, skipping
⚠ Page 4: no new movies, skipping
⚠ Page 5: no new movies, skipping
⚠ Page 6: no new movies, skipping
⚠ Page 7: no new movies, skipping
⚠ Page 8: no new movies, skipping
⚠ Page 9: no new movies, skipping
⚠ Page 10: no new movies, skipping
⚠ Page 11: no new movies, skipping
⚠ Page 12: no new movies, skipping
⚠ Page 13: no new movies, skipping
⚠ Page 14: no new movies, skipping
⚠ Page 15: no new movies, skipping
⚠ Page 16: no new movies, skipping
⚠ Page 17: no new movies, skipping
⚠ Page 18: no new movies, skipping
⚠ Page 19: no new movies, skipping
⚠ Page 20: no new movies, skipping
⚠ Page 21: no new movies, skipping
⚠ Page 22: no new movies, skipping
⚠ Page 23: no new movies, skipping
⚠ Page 24: no new movies, skipping
⚠ Page 25: no new movies, skipping
⚠ Page 26: no new movies, skipping
⚠ Page

> **🛠 Data Engineering Excellence**  
> - Drops duplicates to maintain data integrity  
> - Employs vectorized Pandas operations for speed  
> - Builds genre map from live API for future‑proofing  

In [3]:
# Load raw JSON data from files
raw_folder = Path("data/raw")
all_movies = []
for file in raw_folder.glob("popular_page_*.json"):
    with open(file, encoding='utf-8') as f:
        all_movies.extend(json.load(f))

# Convert to DataFrame
df = pd.DataFrame(all_movies)

# Keep relevant columns and drop duplicates (by unique ID)
# Convert genre_ids list to tuple for consistency
if df['genre_ids'].apply(lambda x: isinstance(x, list)).any():
    df['genre_ids'] = df['genre_ids'].apply(lambda ids: tuple(ids))
df = df[['id', 'title', 'release_date', 'popularity', 'vote_average', 'genre_ids']]
# Drop duplicates based on 'id'
df = df.drop_duplicates(subset=['id'])

# Parse 'release_date' to datetime and drop missing to datetime and drop missing
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df = df.dropna(subset=['release_date'])

# Fetch genre map
genre_resp = requests.get(
    "https://api.themoviedb.org/3/genre/movie/list",
    headers=headers,
    params={'language': 'en-US'}
).json()
genre_map = {g['id']: g['name'] for g in genre_resp['genres']}

# Map genre IDs to names

def map_genres(ids):
    return [genre_map.get(i, "Unknown") for i in ids]

df['genres'] = df['genre_ids'].apply(map_genres)

# Explode 'genres' for per-genre analysis
df_exploded = df.explode('genres')

# Preview cleaned DataFrame
df.head()

,id,title,release_date,popularity,vote_average,genre_ids,genres
0,1241436,Warfare,2025-04-09,485.3223,7.158,"(10752, 28)","[War, Action]"
1,950387,A Minecraft Movie,2025-03-31,409.3111,6.300,"(10751, 35, 12, 14)","[Family, Comedy, Adventure, Fantasy]"
2,977294,Tin Soldier,2025-05-22,383.4020,0.000,"(28, 53)","[Action, Thriller]"
3,1144430,Last Bullet,2025-05-06,377.9668,6.821,"(28, 80, 53)","[Action, Crime, Thriller]"
4,447273,Snow White,2025-03-12,313.0346,4.365,"(10751, 14)","[Family, Fantasy]"


## 📊 4. Interactive & Holistic Exploratory Data Analysis

> In this section, I combine rigorous statistical methods, interactive visualizations, and narrative-driven storytelling to extract actionable insights from 60+ TMDB movies.

- **Multi‑Modal Visualization**: Histograms, boxplots, time‑series, and bar charts—fully interactive with Plotly for real‑time exploration.  
- **Clean Data Pipeline**: End‑to‑end flow from raw API JSON to pandas cleaning, date parsing, dynamic genre mapping, and artifact versioning.  
- **Stakeholder‑Centric**: Clear titles, labeled axes, and “why this matters” annotations that translate data patterns into business impact.  
- **User Empowerment**: Integrated `ipywidgets` slider & dropdown for on‑the‑fly filtering by year and genre—ideal for non‑technical audiences.  
- **Reproducibility & Scalability**: Modular functions and Git‑ignored `data/raw` & `data/processed` directories ensure audit‑friendly, production‑grade workflows.

> *Showcases technical fluency, data storytelling acumen, and best practices for end‑to‑end analytics.*  


### 4.1 ⭐ Rating Distribution  
**🔍 Insightful Distribution Analysis**  
- Combines histogram and boxplot to reveal central tendencies, dispersion, and outliers in movie ratings  
- `nbins=20` balances granularity vs. readability for spotting rating clusters  
- Interactive Plotly visuals allow stakeholders to hover for exact counts and quartiles  


In [4]:

# Configure Plotly renderer to 'notebook_connected' to avoid nbformat errors
import plotly.io as pio
pio.renderers.default = 'notebook_connected'  # alternative renderer
pio.renderers.default = 'iframe_connected'

fig = px.histogram(df, x='vote_average', nbins=20, title='Distribution of Movie Ratings')
fig.show()

fig_box = px.box(df, y='vote_average', title='Boxplot of Movie Ratings')
fig_box.show()

### 4.2 ⏳ Release Trends by Year  
**📈 Temporal Trend Detection**  
- Converts `release_date` to a numeric `year` field for time‑series grouping  
- Line chart highlights production spikes/dips, enabling correlation with industry events  
- Clear axis labels and titles make it presentation‑ready for executive meetings  


In [5]:
df['year'] = df['release_date'].dt.year
time_series = df.groupby('year').size().reset_index(name='count')
fig_ts = px.line(time_series, x='year', y='count', title='Number of Movies Released per Year')
fig_ts.show()

### 4.3 🎯 Genre Popularity  
**📊 Genre Demand Profiling**  
- Explodes multi‑genre lists to capture each film’s full genre footprint  
- Bar chart with categorical x‑axis enforces logical ordering for comparison  
- Identifies high‑value genres to inform content strategy, recommendation engines, and marketing  


In [6]:
genre_counts = df_exploded['genres'].value_counts().reset_index()
genre_counts.columns = ['genre', 'count']
fig_bar = px.bar(genre_counts, x='genre', y='count', title='Movies per Genre')
fig_bar.update_xaxes(type='category')
fig_bar.show()

**💡 User‑Driven Insights**  
- Integrates `ipywidgets` for slider and dropdown filters  
- Separates UI declaration from update logic for clean code  
- Delivers real‑time scatter plots ideal for stakeholder demos or workshops  


In [7]:
# Create widgets
year_slider = widgets.IntRangeSlider(
    value=[df['year'].min(), df['year'].max()],
    min=df['year'].min(), max=df['year'].max(), step=1,
    description='Year Range:'
)
genre_dropdown = widgets.Dropdown(
    options=['All'] + sorted(df_exploded['genres'].unique()),
    value='All', description='Genre:'
)

# Define update function
def update_plots(year_range, genre):
    filtered = df_exploded[(df_exploded['year'] >= year_range[0]) & (df_exploded['year'] <= year_range[1])]
    if genre != 'All':
        filtered = filtered[filtered['genres'] == genre]
    scatter = px.scatter(
        filtered, x='popularity', y='vote_average', hover_data=['title', 'release_date'],
        title=f'Popularity vs Rating ({genre}, {year_range[0]}–{year_range[1]})'
    )
    scatter.show()

# Display interactive controls
ui = widgets.interactive_output(
    update_plots,
    {'year_range': year_slider, 'genre': genre_dropdown}
)
display(widgets.VBox([year_slider, genre_dropdown]), ui)

TypeError: '<' not supported between instances of 'float' and 'str'